In [1]:
using DataFrames, DataFramesMeta, LinearAlgebra, Distributions, Optim, Revise, Plots, CSV, Serialization
using JuMP, Ipopt
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

In [38]:
deltas = deserialize("deltas.jls");
dfj_full = deserialize("dfj.jls");

In [39]:
# # disutility of distance
# β = [-0.5, -1.];
K = 2
nI = 1000
J = nrow(dfj_full)
X = hcat(dfj_full.auxd, dfj_full.auxd2);
Z = hcat(dfj_full.x1, dfj_full.x2);
ζ = rand(Normal(0,1), K, nI);
s = dfj_full.s; #using shares or quantities

In [40]:
Φ1 = Z' * Z
initial_b_guess = [-1., -2.]
b_init = optimize(x -> m.gmm(x, s, X, Z, ζ, Φ1), initial_b_guess, LBFGS()).minimizer
d_init = m.loop(s, X, ζ, b_init);

In [41]:
if any(isnan.(d_init))
    nanind = isnan.(d_init)
    println(sum(nanind))
    dfj = dfj_full[.!nanind,:];
    describe(dfj_full[nanind, :], :mean, :median, :min, :max, cols = [:q, :s, :auxd, :x1, :x2])
else
    dfj = dfj_full
end;
# redefine in case we have NaN
J = nrow(dfj)
X = hcat(dfj.auxd, dfj.auxd2);
Z = hcat(dfj.x1, dfj.x2);

In [42]:
Φ1 = Z' * Z
b_ = optimize(x -> m.gmm(x, s, X, Z, ζ, Φ1), initial_b_guess, LBFGS()).minimizer
d_ = m.loop(s, X, ζ, b_);
println(sum(isnan.(d_)))

0


In [43]:
b_forΦ = m.grad(X, Z, Φ1, d_)

2-element Vector{Float64}:
  15.406324867159128
 -13.74538281688001

In [44]:
ξ_ = d_ - X * b_forΦ;
Φ = Z' * ξ_ * ξ_' * Z

2×2 Matrix{Float64}:
 4.38799e-16  5.26679e-17
 5.26688e-17  6.32176e-18

In [45]:
b = optimize(x -> m.gmm(x, s, X, Z, ζ, Φ), initial_b_guess, LBFGS()).minimizer

2-element Vector{Float64}:
 -1.0
 -2.0

In [46]:
δ = m.loop(s, X, ζ, b, verbose = true);

iterations: 2
dist: 1.4210854715202004e-14
deltas:[-42.37279658055172, -45.80835043555652, -49.07800321381782, -43.36380977101061, -47.51091141515346]
s_:    [0.0166520959999999, 0.002520106499999999, 0.0022313865000000046, 0.003315309799999989, 0.004353770999999998]


In [47]:
β_blp = m.grad(X, Z, Φ, δ)

2-element Vector{Float64}:
  15.406324859708548
 -13.745382811874151

In [110]:
# m.gmm(β, dfj.s, X, Z, ζ)

# (X' * X )*( X'  * deltas)
# model = Model(Ipopt.Optimizer)
# @variable(model, β)
# @NLobjective(model, Min, m.gmm(β, dfj.s, X, Z, ζ))

# this is messed up
# model = Model(Ipopt.Optimizer)
# @variable(model, β)
# register(model, :g, 1, m.gmm, m.grad, autodiff = true)
# @NLobjective(model, Min, (s, X, Z, ζ))
# optimize!(model)
# solution_summary(model; verbose = true)

In [50]:
K = 2
nI = 1000
J = nrow(dfj_full)
X = hcat(dfj_full.auxd, dfj_full.auxd2);
println(size(X))
ζ = rand(Normal(0,1), K, nI);
println(size(ζ))
s = dfj_full.s; #using shares or quantities

(410, 2)
(2, 1000)


In [49]:
δ0 = zeros(1, size(ζ, 2))
δ_ = ones(size(s))
β = [-0.5, -1.];


In [51]:
println(size(ζ .* β))

(2, 1000)


In [56]:
J

410

In [57]:
println("Dims of ζ: ", size(ζ))
println("Dims of ζ .* β: ", size(ζ .* β))
println("Dims of X * (ζ .* β): ", size(X * (ζ .* β)))
println("Dims of δ_ .+ (X * (ζ .* β)): ", size(δ_ .+ (X * (ζ .* β))))
println("Dims of u: ", size(u))
println("Dims of s: ", size(s))

Dims of ζ: (2, 1000)
Dims of ζ .* β: (2, 1000)
Dims of X * (ζ .* β): (410, 1000)
Dims of δ_ .+ (X * (ζ .* β)): (410, 1000)
Dims of u: (411, 1000)
Dims of s: (411, 1)


In [67]:
ζ

2×1000 Matrix{Float64}:
 1.28862  -1.0541    0.261135  -0.780647  …  0.491427  -1.4154   -0.711331
 1.44669   1.07391  -0.072652   2.04786      0.875245   0.69824   0.501537

In [68]:
β

2-element Vector{Float64}:
 -0.5
 -1.0

In [69]:
(ζ .* β)

2×1000 Matrix{Float64}:
 -0.64431   0.52705  -0.130567   0.390323  …  -0.245714   0.707699   0.355665
 -1.44669  -1.07391   0.072652  -2.04786      -0.875245  -0.69824   -0.501537

In [71]:
X

410×2 Matrix{Float64}:
 2.40807  5.7988
 2.44936  5.99935
 2.53129  6.40744
 2.39125  5.71807
 2.50817  6.29093
 2.44839  5.9946
 2.35418  5.54214
 2.53556  6.42907
 2.11898  4.49007
 2.46085  6.0558
 ⋮        
 2.41092  5.81255
 2.52271  6.36407
 2.50257  6.26287
 2.81035  7.89809
 2.33969  5.47415
 2.73808  7.49709
 2.48494  6.17491
 2.4118   5.81676
 2.36637  5.59969

In [70]:
(X * (ζ .* β))

410×1000 Matrix{Float64}:
  -9.94059  -4.9582   0.106879   -10.9352   …  -5.66707  -2.34477  -2.05185
 -10.2573   -5.15181  0.116059   -11.3298      -5.85274  -2.45558  -2.13775
 -10.9005   -5.54688  0.13501    -12.1335      -6.23006  -2.68254  -2.31328
  -9.81295  -4.88036  0.10321    -10.7765      -5.59227  -2.3003   -2.01734
 -10.717    -5.43394  0.129563   -11.904       -6.1224   -2.61755  -2.26307
 -10.2498   -5.14721  0.11584    -11.3205   …  -5.84834  -2.45295  -2.13571
  -9.53456  -4.71097  0.0952693  -10.4307      -5.42919  -2.2037   -1.94229
 -10.9345   -5.56785  0.136024   -12.1762      -6.25004  -2.69463  -2.32261
  -7.861    -3.70511  0.0495432   -8.36796     -4.45057  -1.63555  -1.49829
 -10.3464   -5.20637  0.118659   -11.4409      -5.90497  -2.48686  -2.16197
   ⋮                                        ⋱                      
  -9.96232  -4.97146  0.107506   -10.9623      -5.6798   -2.35235  -2.05773
 -10.8322   -5.50482  0.132979   -12.0481      -6.18998  -2.65833  -2.

In [52]:
u = [δ_ .+ (X * (ζ .* β)); δ0]
# println(mean(u))
e = exp.(u)
s = mean(e, dims=2)

411×1 Matrix{Float64}:
      1.708046652416107e7
      3.6253253963017374e7
      1.677021585973194e8
      1.2616604476584181e7
      1.082987851858634e8
      3.561218389169406e7
      6.521548536488545e6
      1.8188305622789317e8
 127775.00667313361
      4.4807905283273906e7
      ⋮
      1.4250554490995115e8
      9.747243410101286e7
      4.493119217174229e10
      5.053849636319714e6
      9.995867908984142e9
      7.006685984517105e7
      1.8271381911401536e7
      8.092513844217124e6
      1.0